In [86]:
import pandas as pd
from datetime import *
import json
import yfinance

df = pd.read_csv('spx2020.csv')
df = df.drop(columns=['Unnamed: 0'])
df['# of earnings'] = -1
df['% of positive impact of earnings'] = -1.0
df['# of positive surprise earnings'] = -1
df['% of positive impact of earnings when positive surprise'] = -1.0
df['# of negative surprise earnings'] = -1
df['% of positive impact of earnings when negative surprise'] = -1.0
df

,ticker,company,# of earnings,% of positive impact of earnings,# of positive surprise earnings,% of positive impact of earnings when positive surprise,# of negative surprise earnings,% of positive impact of earnings when negative surprise
0,VIAC,VIACOMCBS INC,-1,-1.0,-1,-1.0,-1,-1.0
1,SCHW,SCHWAB CHARLES CORP NEW,-1,-1.0,-1,-1.0,-1,-1.0
2,CSCO,CISCO SYSTEMS INC,-1,-1.0,-1,-1.0,-1,-1.0
3,HD,HOME DEPOT INC,-1,-1.0,-1,-1.0,-1,-1.0
4,QCOM,QUALCOMM INC,-1,-1.0,-1,-1.0,-1,-1.0
...,...,...,...,...,...,...,...,...
499,FOX,FOX CORP,-1,-1.0,-1,-1.0,-1,-1.0
500,GOOG,GOOGLE INC,-1,-1.0,-1,-1.0,-1,-1.0
501,GOOGL,GOOGLE INC,-1,-1.0,-1,-1.0,-1,-1.0
502,UA,UNDER ARMOUR INC,-1,-1.0,-1,-1.0,-1,-1.0


In [87]:
for i in range(len(df)):
    stock_abbr = df.iloc[i, 0]
    print(stock_abbr)
    !node get_earnings2.js {stock_abbr}
    f = open(f'{stock_abbr}.json')
    earnings = json.load(f)
    f.close()
    !rm {stock_abbr}.json

    start, end = '1995-01-01', '2022-12-31'
    stock_ticker = yfinance.Ticker(stock_abbr)
    stock = stock_ticker.history(start=start, end=end)
    stock['Close_t+1'] = stock.Close.shift(-1)

    count, count_up = 0, 0
    count_pos, count_pos_up = 0, 0
    count_neg, count_neg_up = 0, 0
    for j in range(len(earnings)):
        if earnings[j]['eps'] and earnings[j]['date'] in stock.index:
            count += 1
            if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                count_up += 1
            if earnings[j]['estimated_eps']:
                if earnings[j]['estimated_eps'] < earnings[j]['eps']:
                    count_pos += 1
                    if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                        count_pos_up += 1
                elif earnings[j]['estimated_eps'] > earnings[j]['eps']:
                    count_neg += 1
                    if stock.loc[earnings[j]['date'], 'Close_t+1'] > stock.loc[earnings[j]['date'], 'Close']:
                        count_neg_up += 1
    pos_impact = count_up / count if count > 0 else -1
    pos_pos_impact = count_pos_up / count_pos if count_pos > 0 else -1
    neg_pos_impact = count_neg_up / count_neg if count_neg > 0 else -1
    df.at[i, '# of earnings'] = count
    df.at[i, '% of positive impact of earnings'] = pos_impact
    df.at[i, '# of positive surprise earnings'] = count_pos
    df.at[i, '% of positive impact of earnings when positive surprise'] = pos_pos_impact
    df.at[i, '# of negative surprise earnings'] = neg_pos_impact
    df.at[i, '% of positive impact of earnings when negative surprise'] = neg_pos_impact
    print(f'earnings count: {count}')
    print(f'positive impact: {pos_impact}')
    print(f'positive surprise count: {count_pos}')
    print(f'positive impact on positive surprise: {pos_pos_impact}')
    print(f'negative surprise count: {count_neg}')
    print(f'positive impact on negative surprise: {neg_pos_impact}\n')

VIAC
- VIAC: No data found, symbol may be delisted
earnings count: 0
positive impact: -1
positive surprise count: 0
positive impact on positive surprise: -1
negative surprise count: 0
positive impact on negative surprise: -1

SCHW
earnings count: 112
positive impact: 0.5089285714285714
positive surprise count: 45
positive impact on positive surprise: 0.5333333333333333
negative surprise count: 45
positive impact on negative surprise: 0.2631578947368421

CSCO
earnings count: 112
positive impact: 0.5178571428571429
positive surprise count: 92
positive impact on positive surprise: 0.532608695652174
negative surprise count: 92
positive impact on negative surprise: 0.0

HD
earnings count: 112
positive impact: 0.4642857142857143
positive surprise count: 80
positive impact on positive surprise: 0.4375
negative surprise count: 80
positive impact on negative surprise: 0.6666666666666666

QCOM
earnings count: 110
positive impact: 0.5
positive surprise count: 76
positive impact on positive surpri

In [83]:
start, end = '1995-01-01', '2022-12-31'
stock_ticker = yfinance.Ticker('XLNX')
stock = stock_ticker.history(start=start, end=end)
stock

- XLNX: No data found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [84]:
stock_abbr = 'XLNX'
!node get_earnings2.js {stock_abbr}
f = open(f'{stock_abbr}.json')
earnings = json.load(f)
print(earnings)
f.close()
!rm {stock_abbr}.json

[{'date': '2022-01-26', 'eps': 1.19, 'estimated_eps': 0.95, 'timing': 2}, {'date': '2021-10-27', 'eps': 0.94, 'estimated_eps': 0.87, 'timing': 2}, {'date': '2021-07-28', 'eps': 0.83, 'estimated_eps': 0.77, 'timing': 2}, {'date': '2021-05-04', 'eps': 0.75, 'estimated_eps': 0.71, 'timing': 2}, {'date': '2021-01-27', 'eps': 0.69, 'estimated_eps': 0.68, 'timing': 2}, {'date': '2020-10-21', 'eps': 0.79, 'estimated_eps': 0.74, 'timing': 2}, {'date': '2020-07-30', 'eps': 0.38, 'estimated_eps': 0.56, 'timing': 2}, {'date': '2020-04-22', 'eps': 0.75, 'estimated_eps': 0.69, 'timing': 2}, {'date': '2020-01-28', 'eps': 0.64, 'estimated_eps': 0.59, 'timing': 2}, {'date': '2019-10-23', 'eps': 0.89, 'estimated_eps': 0.9, 'timing': 2}, {'date': '2019-07-24', 'eps': 0.94, 'estimated_eps': 0.94, 'timing': 2}, {'date': '2019-04-24', 'eps': 0.95, 'estimated_eps': 0.94, 'timing': 2}, {'date': '2019-01-23', 'eps': 0.93, 'estimated_eps': 0.85, 'timing': 2}, {'date': '2018-10-24', 'eps': 0.84, 'estimated_eps'

In [69]:
print(df.to_string())

    ticker                           company  # of earnings  % of positive impact of earnings
0     VIAC                     VIACOMCBS INC              0                         -1.000000
1     SCHW           SCHWAB CHARLES CORP NEW            112                          0.508929
2     CSCO                 CISCO SYSTEMS INC            112                          0.517857
3       HD                    HOME DEPOT INC            112                          0.464286
4     QCOM                      QUALCOMM INC            110                          0.500000
5      AES                        A E S CORP            111                          0.549550
6      UNM                        UNUM GROUP            112                          0.544643
7        C                     CITIGROUP INC            112                          0.589286
8       MS   MORGAN STANLEY DEAN WITTER & CO            102                          0.558824
9      AZO                      AUTOZONE INC            108 

In [79]:
df.to_csv('earnings_impact3.csv', index=False)